In [ ]:
# Import necessary libraries and modules.
# Authenticate to access Earth Engine.
# Initialize the Earth Engine module.
# Load the uploaded shapefile containing the updated boundaries for both districts and upazilas.
# Filter for a specific district using the shapefile.
# Load Sentinel-2 surface reflectance data.
# Define a function to calculate ARVI.
# Map the ARVI calculation function over the image collection.
# Create a composite image of the median ARVI.
# Calculate the minimum and maximum ARVI values in the clipped region.
# Define legend keys based on dynamic min and max values.
# Define your color codes.
# Create a color map using the color codes.
# Generate gradient colors.
# Display the gradient colors.
# Generate a gradient using the color map for visualization.
# Plot the gradient.
# Define a function to get the highest ARVI and coordinates for each upazila.
# Apply the function to each upazila within the specific district.
# Select relevant properties for each upazila to a CSV file.
# Export the data to a CSV file in Google Drive.
# Add layers to the map.
# Center the map on the specified district.
# Define a function to add markers for each upazila with the highest ARVI.
# Add markers for each upazila.
# Add a dynamic legend to the map.
# Display the map.


import ee
from google.colab import auth
import geemap
from ipyleaflet import Marker, Popup, WidgetControl
from ipywidgets import HTML
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

# Authenticate to access Earth Engine
auth.authenticate_user()

# Initialize the Earth Engine module
ee.Initialize(project='ee-2june2024')

# Load the uploaded shapefile containing the updated boundaries for both districts and upazilas
boundaries2020 = ee.FeatureCollection("projects/ee-2june2024/assets/bgd_adm_03")

# Filter for a specific district using the shapefile
district_name = 'Dhaka'  # Specify the district name here
specific_district = boundaries2020.filter(ee.Filter.eq('district', district_name))

# Load Sentinel-2 surface reflectance data
s2_sr = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(specific_district.geometry()) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Function to calculate ARVI
def calculate_arvi(image):
    arvi = image.expression(
        '((NIR - (2 * RED - BLUE)) / (NIR + (2 * RED - BLUE)))',
        {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }
    ).rename('ARVI')
    return image.addBands(arvi)

# Map the function over the image collection
s2_sr_arvi = s2_sr.map(calculate_arvi)

# Create a composite image of the median ARVI
arvi_composite = s2_sr_arvi.select('ARVI').median().clip(specific_district.geometry())

# Calculate the minimum and maximum ARVI values in the clipped region
min_max = arvi_composite.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=specific_district.geometry(),
    scale=10,
    maxPixels=1e9
)
min_value = min_max.get('ARVI_min').getInfo() if min_max.get('ARVI_min') else -1
max_value = min_max.get('ARVI_max').getInfo() if min_max.get('ARVI_max') else 1

# Define legend keys based on dynamic min and max values
num_legend_items = 11  # Number of legend items
step = (max_value - min_value) / (num_legend_items - 1)
legend_keys = [f"{min_value + i * step:.2f}" for i in range(num_legend_items)]

# Define your color codes
color_codes = ['#097e41', '#42ad59', '#8acf66', '#c6e880', '#f3faad', '#f6ba6f', '#f47343', '#d43023', '#a20126']

# Create a color map using the color codes
def generate_gradient_colors(num_colors, colors):
    cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', colors, N=num_colors)
    gradient_colors = [mcolors.rgb2hex(cmap(i / num_colors)) for i in range(num_colors)]
    return gradient_colors

# Generate gradient colors
gradient_colors = generate_gradient_colors(num_legend_items, color_codes)

# Display the gradient colors
print(gradient_colors)

# Generate a gradient using the color map for visualization
cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', gradient_colors, N=256)
gradient = np.linspace(0, 1, 256).reshape(1, -1)
gradient = np.vstack((gradient, gradient))

# Plot the gradient
fig, ax = plt.subplots(figsize=(8, 2))
ax.imshow(gradient, aspect='auto', cmap=cmap)
ax.set_axis_off()
plt.show()

# Function to get the highest ARVI and coordinates for each upazila
def get_max_intensity(feature):
    max_value = arvi_composite.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=feature.geometry(),
        scale=10,
        maxPixels=1e9
    ).get('ARVI')

    max_value_image = ee.Image.constant(max_value)

    max_points = arvi_composite.eq(max_value_image).selfMask().reduceToVectors(
        geometry=feature.geometry(),
        scale=10,
        maxPixels=1e9,
        geometryType='centroid',
        eightConnected=False
    )

    return feature.set({
        'max_intensity': max_value,
        'max_points': max_points.geometry().coordinates()
    })

# Apply the function to each upazila within the specific district
district_with_intensity = specific_district.map(get_max_intensity)

# Export relevant properties for each upazila to a CSV file
export_data = district_with_intensity.select(['upazila', 'max_intensity', 'max_points'])

# Export the data to a CSV file in Google Drive
task = ee.batch.Export.table.toDrive(
    collection=export_data,
    description=f'{district_name}_Upazila_level_ARVI',
    fileFormat='CSV',
    selectors=['upazila', 'max_intensity', 'max_points']
)
task.start()

# Add layers to the map
Map = geemap.Map()
Map.addLayer(specific_district, {}, 'Dhaka Upazilas')
Map.addLayer(arvi_composite, {'min': min_value, 'max': max_value, 'palette': gradient_colors}, 'ARVI')

# Center map on the specified district
Map.centerObject(specific_district, 10)

# Add markers for each upazila with highest ARVI
def add_markers(feature):
    max_points = feature.get('max_points').getInfo()
    intensity = feature.get('max_intensity').getInfo()
    upazila_name = feature.get('upazila').getInfo()

    if isinstance(max_points[0], list):  # Check if it's nested
        for point in max_points:
            lon, lat = point
            point_geom = ee.Geometry.Point(lon, lat)
            if specific_district.geometry().contains(point_geom).getInfo():
                marker = Marker(location=(lat, lon))
                popup = Popup(
                    location=(lat, lon),
                    child=HTML(f"<b>Upazila:</b> {upazila_name}<br><b>Max ARVI:</b> {intensity}<br><b>Coordinates:</b> ({lat:.6f}, {lon:.6f})"),
                    close_button=False,
                    auto_close=False,
                    close_on_escape_key=False
                )
                Map.add_layer(marker)
                Map.add_layer(popup)
    else:
        lon, lat = max_points
        point_geom = ee.Geometry.Point(lon, lat)
        if specific_district.geometry().contains(point_geom).getInfo():
            marker = Marker(location=(lat, lon))
            popup = Popup(
                location=(lat, lon),
                child=HTML(f"<b>Upazila:</b> {upazila_name}<br><b>Max ARVI:</b> {intensity}<br><b>Coordinates:</b> ({lat:.6f}, {lon:.6f})"),
                close_button=False,
                auto_close=False,
                close_on_escape_key=False
            )
            Map.add_layer(marker)
            Map.add_layer(popup)

# Get the list of features and add markers for each
features = district_with_intensity.getInfo()['features']
for feature in features:
    add_markers(ee.Feature(feature))

# Add dynamic legend to the map
legend_dict = {legend_keys[i]: gradient_colors[i] for i in range(num_legend_items)}
Map.add_legend(title="ARVI", legend_dict=legend_dict, position="bottomleft")

# Display the map
Map
